In [2]:
import pandas as pd

#Reading raw data
data = pd.read_csv('full_data_may22.csv',encoding = "ISO-8859-1")

In [3]:
#Extracting only EUIs and SIBs
data =  data.loc[data['EUI_OR_SIB'].isin(['EUI','SIB'])]

In [4]:
data_sus=data[data['HOUR_TO_COMPLETION'].isnull()]

#Extracting out NaNs in 'Hour To Completion'
data_sus.to_csv('nans.csv', index=False)

In [65]:
#Extracting relevant variables
data_analy = data[["ISSUE_KEY","PLATFORM","PRODUCT_FAMILY","CREATED_TIMESTAMP","SEVERITY", "GROUP_NAMES_SEPARATED","COE_NAMES_SEPARATED_COPY","RESOLUTION_TIME","HOUR_TO_COMPLETION","TEAMS_1"]]

In [66]:
data_analy['HOUR_TO_COMPLETION'] = data_analy['HOUR_TO_COMPLETION'].replace(0,1) 
#Replacing zero with one so to avoid inf values

C:\Users\User\AppData\Local\Temp/ipykernel_9848/909106011.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_analy['HOUR_TO_COMPLETION'] = data_analy['HOUR_TO_COMPLETION'].replace(0,1)


In [67]:
#Removing NA Values
data_analy = data_analy[data_analy['COE_NAMES_SEPARATED_COPY'].notna()]
data_analy = data_analy[data_analy['GROUP_NAMES_SEPARATED'].notna()]
data_analy = data_analy[data_analy['RESOLUTION_TIME'].notna()]
data_analy = data_analy[data_analy['HOUR_TO_COMPLETION'].notna()]

In [69]:
#Calculation of Teams Variables.

import numpy as np
import statistics as st

k=0
odds=[]
bre=[]
dev=[]
rows = []
for m in months_names:
    x=data_analy.loc[(data_analy["CREATED_TIMESTAMP"] == m)]
    team_names=x['TEAMS_1'].unique()
    for t in team_names:
        y=x.loc[x['TEAMS_1']== t]
        eui_tot=len(y)
        sev_names=y['SEVERITY'].unique()
        sev_names.sort()
        for s in sev_names:
            z=y.loc[y['SEVERITY']== s]
            eui_s=(len(z)/len(y))*100
            for i in range(0,len(z)):
                odd=(z.iloc[i,8])/(z.iloc[i,7])
                odds.append(odd)
                if z.iloc[i,8] > z.iloc[i,7]:
                    k=k+1
            bre=(k/len(z))
            dev=st.median(odds)
            grou= z.iloc[i,5]
            coe= z.iloc[i,6]
            wei= int(s.replace('P',""))
            wei=5-wei
            rows.append([m,t,s,eui_s,bre,dev,wei,grou,coe])
            k=0
            odds=[]

df_months_team = pd.DataFrame(rows, columns=["Month", "Team","Severity","EUI Percentage","Breach","Deviation","Weight","Group","COE"])

In [70]:
# Normalizing all variables
from sklearn import preprocessing
df_months_team_stand=df_months_team
min_max_scaler = preprocessing.MinMaxScaler()
df_months_team_stand.iloc[:,3:7] = min_max_scaler.fit_transform(df_months_team_stand.iloc[:,3:7])

In [71]:
#Teams Health Score
HS_COE=0
df_teams = pd.DataFrame(columns = months_names)
for m in months_names:
    x=df_months_team_stand.loc[df_months_team["Month"] == m]
    team_names=x['Team'].unique()
    for t in team_names:
        y=x.loc[x['Team']== t]
        for i in range(0,len(y)):
            hs=((0.25*y.iloc[i,3])+(0.25*y.iloc[i,6])+(0.25*y.iloc[i,5])+(0.25*y.iloc[i,4]))
            HS_COE=HS_COE+hs
        df_teams.at[t,m] = 1/(HS_COE)
        HS_COE=0

In [72]:
#Transforming to buckets and filling NA values with 4
df_teams=df_teams.fillna(4)
df_teams=round(df_teams)

In [73]:
#Calculation of Teams Variables, By platforms

import numpy as np
import statistics as st

k=0
odds=[]
bre=[]
dev=[]
rows = []
for m in months_names:
    x=data_analy.loc[(data_analy["CREATED_TIMESTAMP"] == m)]
    team_names=x['TEAMS_1'].unique()
    for t in team_names:
        y=x.loc[x['TEAMS_1']== t]
        eui_tot=len(y)
        sev_names=y['SEVERITY'].unique()
        sev_names.sort()
        plt_names=y['PLATFORM'].unique()
        for p in plt_names:
            for s in sev_names:
                z=y.loc[y['SEVERITY']== s]
                eui_s=(len(z)/len(y))*100
                for i in range(0,len(z)):
                    odd=(z.iloc[i,8])/(z.iloc[i,7])
                    odds.append(odd)
                    if z.iloc[i,8] > z.iloc[i,7]:
                        k=k+1
                bre=(k/len(z))
                dev=st.median(odds)
                grou= z.iloc[i,5]
                coe= z.iloc[i,6]
                wei= int(s.replace('P',""))
                wei=5-wei
                rows.append([m,t,s,eui_s,bre,dev,wei,p,grou,coe])
                k=0
                odds=[]

df_months_team_plat = pd.DataFrame(rows, columns=["Month", "Team","Severity","EUI Percentage","Breach","Deviation","Weight","Platform","Group","COE"])


In [74]:
# Ignoring all EUIs with no platform (for now)
df_months_team_plat.isnull().sum(axis = 0)
df_months_team_plat=df_months_team_plat.fillna('None')
df_months_team_plat.isnull().sum(axis = 0)

Month             0
Team              0
Severity          0
EUI Percentage    0
Breach            0
Deviation         0
Weight            0
Platform          0
Group             0
COE               0
dtype: int64

In [75]:
#Normalizing team data by platform
from sklearn import preprocessing
df_months_team_plat_stand=df_months_team_plat
min_max_scaler = preprocessing.MinMaxScaler()
df_months_team_plat_stand.iloc[:,3:7] = min_max_scaler.fit_transform(df_months_team_plat_stand.iloc[:,3:7])

In [76]:
###### Teams Health Score by Platforms

#Web
df_teams_web = pd.DataFrame(columns = months_names)
HS_COE=0

df_months_team_plat_stand_web = pd.DataFrame(columns = months_names)
df_months_team_plat_stand_web = df_months_team_plat_stand[df_months_team_plat_stand['Platform'].str.contains('Web')]
for m in months_names:
    x=df_months_team_plat_stand_web.loc[df_months_team_plat_stand_web["Month"] == m]
    team_names=x['Team'].unique()
    for t in team_names:
        y=x.loc[x['Team']== t]
        for i in range(0,len(y)):
            hs=((0.25*y.iloc[i,3])+(0.25*y.iloc[i,6])+(0.25*y.iloc[i,5])+(0.25*y.iloc[i,4]))
            HS_COE=HS_COE+hs
        df_teams_web.at[t,m] = 1/(HS_COE)
        HS_COE=0


#iOS
df_teams_ios = pd.DataFrame(columns = months_names)
HS_COE=0

df_months_team_plat_stand_ios = pd.DataFrame(columns = months_names)
df_months_team_plat_stand_ios = df_months_team_plat_stand[df_months_team_plat_stand['Platform'].str.contains('iOS')]
for m in months_names:
    x=df_months_team_plat_stand_ios.loc[df_months_team_plat_stand_ios["Month"] == m]
    team_names=x['Team'].unique()
    for t in team_names:
        y=x.loc[x['Team']== t]
        for i in range(0,len(y)):
            hs=((0.25*y.iloc[i,3])+(0.25*y.iloc[i,6])+(0.25*y.iloc[i,5])+(0.25*y.iloc[i,4]))
            HS_COE=HS_COE+hs
        df_teams_ios.at[t,m] = 1/(HS_COE)
        HS_COE=0


#Android
df_teams_android = pd.DataFrame(columns = months_names)
HS_COE=0

df_months_team_plat_stand_android = pd.DataFrame(columns = months_names)
df_months_team_plat_stand_android = df_months_team_plat_stand[df_months_team_plat_stand['Platform'].str.contains('Android')]
for m in months_names:
    x=df_months_team_plat_stand_android.loc[df_months_team_plat_stand_android["Month"] == m]
    team_names=x['Team'].unique()
    for t in team_names:
        y=x.loc[x['Team']== t]
        for i in range(0,len(y)):
            hs=((0.25*y.iloc[i,3])+(0.25*y.iloc[i,6])+(0.25*y.iloc[i,5])+(0.25*y.iloc[i,4]))
            HS_COE=HS_COE+hs
        df_teams_android.at[t,m] = 1/(HS_COE)
        HS_COE=0


#Windows
df_teams_wind = pd.DataFrame(columns = months_names)
HS_COE=0

df_months_team_plat_stand_wind = pd.DataFrame(columns = months_names)
df_months_team_plat_stand_wind = df_months_team_plat_stand[df_months_team_plat_stand['Platform'].str.contains('Windows')]
for m in months_names:
    x=df_months_team_plat_stand_wind.loc[df_months_team_plat_stand_wind["Month"] == m]
    team_names=x['Team'].unique()
    for t in team_names:
        y=x.loc[x['Team']== t]
        for i in range(0,len(y)):
            hs=((0.25*y.iloc[i,3])+(0.25*y.iloc[i,6])+(0.25*y.iloc[i,5])+(0.25*y.iloc[i,4]))
            HS_COE=HS_COE+hs
        df_teams_wind.at[t,m] = 1/(HS_COE)
        HS_COE=0


#Backend
df_teams_back = pd.DataFrame(columns = months_names)
HS_COE=0

df_months_team_plat_stand_back = pd.DataFrame(columns = months_names)
df_months_team_plat_stand_back = df_months_team_plat_stand[df_months_team_plat_stand['Platform'].str.contains('Backend')]
for m in months_names:
    x=df_months_team_plat_stand_back.loc[df_months_team_plat_stand_back["Month"] == m]
    team_names=x['Team'].unique()
    for t in team_names:
        y=x.loc[x['Team']== t]
        for i in range(0,len(y)):
            hs=((0.25*y.iloc[i,3])+(0.25*y.iloc[i,6])+(0.25*y.iloc[i,5])+(0.25*y.iloc[i,4]))
            HS_COE=HS_COE+hs
        df_teams_back.at[t,m] = 1/(HS_COE)
        HS_COE=0

In [77]:
#Final Health Score Calculation

#Web
df_teams_web=df_teams_web.fillna(4)
df_teams_web=round(df_teams_web)

#iOS
df_teams_ios=df_teams_ios.fillna(4)
df_teams_ios=round(df_teams_ios)

#Android
df_teams_android=df_teams_android.fillna(4)
df_teams_android=round(df_teams_android)

#Windows
df_teams_wind=df_teams_wind.fillna(4)
df_teams_wind=round(df_teams_wind)

#Backend
df_teams_back=df_teams_back.fillna(4)
df_teams_back=round(df_teams_back)


In [78]:
#Calculation of Teams Variables, By Product

import numpy as np
import statistics as st

k=0
odds=[]
bre=[]
dev=[]
rows = []
for m in months_names:
    x=data_analy.loc[(data_analy["CREATED_TIMESTAMP"] == m)]
    team_names=x['TEAMS_1'].unique()
    for t in team_names:
        y=x.loc[x['TEAMS_1']== t]
        eui_tot=len(y)
        sev_names=y['SEVERITY'].unique()
        sev_names.sort()
        product_names=y['PRODUCT_FAMILY'].unique()
        for prod in product_names:
            for s in sev_names:
                z=y.loc[y['SEVERITY']== s]
                eui_s=(len(z)/len(y))*100
                for i in range(0,len(z)):
                    odd=(z.iloc[i,8])/(z.iloc[i,7])
                    odds.append(odd)
                    if z.iloc[i,8] > z.iloc[i,7]:
                        k=k+1
                bre=(k/len(z))
                dev=st.median(odds)
                grou= z.iloc[i,5]
                coe= z.iloc[i,6]
                wei= int(s.replace('P',""))
                wei=5-wei
                rows.append([m,t,s,eui_s,bre,dev,wei,prod,grou,coe])
                k=0
                odds=[]

df_months_team_prod = pd.DataFrame(rows, columns=["Month", "Team","Severity","EUI Percentage","Breach","Deviation","Weight","Product","Group","COE"])

In [79]:
#Ignoring all EUIs with no product (for now)
df_months_team_prod.isnull().sum(axis = 0)
df_months_team_prod=df_months_team_prod.fillna('None')
df_months_team_prod.isnull().sum(axis = 0)

Month             0
Team              0
Severity          0
EUI Percentage    0
Breach            0
Deviation         0
Weight            0
Product           0
Group             0
COE               0
dtype: int64

In [80]:
#Normalizing data by platform
from sklearn import preprocessing
df_months_team_prod_stand=df_months_team_prod
min_max_scaler = preprocessing.MinMaxScaler()
df_months_team_prod_stand.iloc[:,3:7] = min_max_scaler.fit_transform(df_months_team_prod_stand.iloc[:,3:7])

In [81]:
#Teams Health Score by Products

#ACC
df_teams_acc = pd.DataFrame(columns = months_names)
HS_COE=0

df_months_team_prod_stand_acc = pd.DataFrame(columns = months_names)
df_months_team_prod_stand_acc = df_months_team_prod_stand[df_months_team_prod_stand['Product'].str.contains('ACC')]
for m in months_names:
    x=df_months_team_prod_stand_acc.loc[df_months_team_prod_stand_acc["Month"] == m]
    team_names=x['Team'].unique()
    for t in team_names:
        y=x.loc[x['Team']== t]
        for i in range(0,len(y)):
            hs=((0.25*y.iloc[i,3])+(0.25*y.iloc[i,6])+(0.25*y.iloc[i,5])+(0.25*y.iloc[i,4]))
            HS_COE=HS_COE+hs
        df_teams_acc.at[t,m] = 1/(HS_COE)
        HS_COE=0


#BIM 360
df_teams_bim = pd.DataFrame(columns = months_names)
HS_COE=0

df_months_team_prod_stand_bim = pd.DataFrame(columns = months_names)
df_months_team_prod_stand_bim = df_months_team_prod_stand[df_months_team_prod_stand['Product'].str.contains('BIM 360')]
for m in months_names:
    x=df_months_team_prod_stand_bim.loc[df_months_team_prod_stand_bim["Month"] == m]
    team_names=x['Team'].unique()
    for t in team_names:
        y=x.loc[x['Team']== t]
        for i in range(0,len(y)):
            hs=((0.25*y.iloc[i,3])+(0.25*y.iloc[i,6])+(0.25*y.iloc[i,5])+(0.25*y.iloc[i,4]))
            HS_COE=HS_COE+hs
        df_teams_bim.at[t,m] = 1/(HS_COE)
        HS_COE=0


#PlanGrid
df_teams_pg = pd.DataFrame(columns = months_names)
HS_COE=0

df_months_team_prod_stand_pg = pd.DataFrame(columns = months_names)
df_months_team_prod_stand_pg = df_months_team_prod_stand[df_months_team_prod_stand['Product'].str.contains('PlanGrid')]
for m in months_names:
    x=df_months_team_prod_stand_pg.loc[df_months_team_prod_stand_pg["Month"] == m]
    team_names=x['Team'].unique()
    for t in team_names:
        y=x.loc[x['Team']== t]
        for i in range(0,len(y)):
            hs=((0.25*y.iloc[i,3])+(0.25*y.iloc[i,6])+(0.25*y.iloc[i,5])+(0.25*y.iloc[i,4]))
            HS_COE=HS_COE+hs
        df_teams_pg.at[t,m] = 1/(HS_COE)
        HS_COE=0

In [82]:
#Final Health Score Calculation

#ACC
df_teams_acc=df_teams_acc.fillna(4)
df_teams_acc=round(df_teams_acc)

#BIM 360
df_teams_bim=df_teams_bim.fillna(4)
df_teams_bim=round(df_teams_bim)

#PlanGrid
df_teams_pg=df_teams_pg.fillna(4)
df_teams_pg=round(df_teams_pg)

In [83]:
# Calculating Group Health Score by Teams' Average Health Score
groups=data_analy['GROUP_NAMES_SEPARATED'].unique()
data_analy=data_analy.fillna('None')

In [84]:
df_group = pd.DataFrame(columns = months_names)
for g in groups:
    xa=data_analy.loc[(data_analy['GROUP_NAMES_SEPARATED']==g)]
    rowt=xa['TEAMS_1'].unique()
    if 'None' in rowt:
        rowt=rowt.tolist()
        rowt.remove('None')
    df_group.at[g,:]=df_teams.loc[rowt,:].mean(axis=0)

In [85]:
df_group=round(df_group)

In [86]:
# Calculating COE Health Score by Teams' Average Health Score

coes=data_analy['COE_NAMES_SEPARATED_COPY'].unique()

df_coe = pd.DataFrame(columns = months_names)
for c in coes:
    xa=data_analy.loc[(data_analy['COE_NAMES_SEPARATED_COPY']==c)]
    rowt=xa['GROUP_NAMES_SEPARATED'].unique()
    if 'None' in rowt:
        rowt=rowt.tolist()
        rowt.remove('None')
    df_coe.at[c,:]=df_group.loc[rowt,:].mean(axis=0)

In [87]:
df_coe=round(df_coe)